<a href="https://colab.research.google.com/github/fabrizioca2/LAB_Laberinto/blob/Fabrizio-Calixto/Laberinto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pygame
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [50]:
# Clase que representa el laberinto
class Maze:
    def __init__(self, width, height, obstacles=None):
        """
        Constructor para inicializar el laberinto.
        :param width: Ancho del laberinto (número de columnas).
        :param height: Alto del laberinto (número de filas).
        :param obstacles: Lista de tuplas que representan las coordenadas de las celdas ocupadas.
        """
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        if obstacles:
            for obs in obstacles:
                self.grid[obs[1]][obs[0]] = 1  # Marca las celdas ocupadas

    def is_cell_valid(self, x, y):
        """
        Verifica si una celda es válida y está libre.
        :param x: Coordenada x (columna).
        :param y: Coordenada y (fila).
        :return: True si la celda es válida y libre, False de lo contrario.
        """
        return 0 <= x < self.width and 0 <= y < self.height and self.grid[y][x] == 0

In [51]:
# Clase que representa el algoritmo genético
class GeneticAlgorithm:
    def __init__(self, maze, start, end, population_size, max_generations, crossover_rate, mutation_rate):
        """
        Constructor que inicializa el algoritmo genético.
        """
        self.maze = maze
        self.start = start
        self.end = end
        self.population_size = population_size
        self.max_generations = max_generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = []

    def generate_initial_population(self):
        """
        Genera una población inicial de caminos aleatorios.
        """
        self.population = []
        for _ in range(self.population_size):
            path = [self.start]
            current_pos = self.start
            while current_pos != self.end:
                next_step = (current_pos[0] + random.choice([-1, 0, 1]), current_pos[1] + random.choice([-1, 0, 1]))
                if (self.maze.is_cell_valid(next_step[0], next_step[1]) and
                    next_step not in path):
                    path.append(next_step)
                    current_pos = next_step
            self.population.append(path)

    def fitness_function(self, path):
        """
        Calcula el fitness de un camino.
        """
        if not path or path[-1] != self.end:
            return float('-inf')  # Penalizar caminos que no llegan al destino
        path_length = len(path)
        turns = sum(1 for i in range(1, len(path) - 1) if path[i-1][0] != path[i][0] and path[i][1] != path[i+1][1])
        return 1 / (path_length + turns)

    def selection(self):
        """
        Selecciona los padres de la población usando el método de ruleta.
        """
        fitness_values = [self.fitness_function(path) for path in self.population]
        total_fitness = sum(fitness_values)
        probabilities = [f / total_fitness for f in fitness_values if f > 0]
        return random.choices(self.population, weights=probabilities, k=self.population_size)

    def crossover(self, parent1, parent2):
        """
        Realiza el cruce entre dos caminos para generar dos nuevos.
        """
        if random.random() < self.crossover_rate:
            cross_point = random.randint(1, min(len(parent1), len(parent2)) - 2)
            child1 = parent1[:cross_point] + [p for p in parent2 if p not in parent1[:cross_point]]
            child2 = parent2[:cross_point] + [p for p in parent1 if p not in parent2[:cross_point]]
            return child1, child2
        return parent1, parent2

    def mutate(self, path):
        """
        Realiza una mutación en un camino.
        """
        if random.random() < self.mutation_rate and len(path) > 2:
            mutation_point = random.randint(1, len(path) - 2)
            new_step = (path[mutation_point][0] + random.choice([-1, 0, 1]), path[mutation_point][1] + random.choice([-1, 0, 1]))
            if self.maze.is_cell_valid(new_step[0], new_step[1]) and new_step not in path:
                path[mutation_point] = new_step
        return path

    def evolve(self):
        """
        Ejecuta el proceso evolutivo para encontrar el mejor camino.
        """
        self.generate_initial_population()
        best_path = []
        for generation in range(self.max_generations):
            parents = self.selection()
            next_generation = []
            for i in range(0, len(parents) - 1, 2):
                parent1, parent2 = parents[i], parents[i + 1]
                child1, child2 = self.crossover(parent1, parent2)
                next_generation.append(self.mutate(child1))
                next_generation.append(self.mutate(child2))
            self.population = sorted(next_generation, key=self.fitness_function, reverse=True)[:self.population_size]
            best_path = self.population[0]
            if best_path[-1] == self.end:
                break
        return best_path

In [52]:
# Visualización con pygame
def draw_maze(maze, path):
    pygame.init()
    cell_size = 40
    screen = pygame.display.set_mode((maze.width * cell_size, maze.height * cell_size))
    pygame.display.set_caption("Laberinto Resuelto con Algoritmo Genético")
    clock = pygame.time.Clock()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        screen.fill((255, 255, 255))
        for y in range(maze.height):
            for x in range(maze.width):
                color = (0, 0, 0) if maze.grid[y][x] == 1 else (200, 200, 200)
                pygame.draw.rect(screen, color, (x * cell_size, y * cell_size, cell_size, cell_size))
                pygame.draw.rect(screen, (0, 0, 0), (x * cell_size, y * cell_size, cell_size, cell_size), 1)

        for step in path:
            pygame.draw.rect(screen, (0, 255, 0), (step[0] * cell_size, step[1] * cell_size, cell_size, cell_size))

        pygame.display.flip()
        clock.tick(10)

    pygame.quit()

In [55]:
# Crear el laberinto y resolverlo
maze = Maze(10, 10, obstacles=[(2, 2), (3, 3), (4, 4), (5, 5), (6, 6)])
solver = GeneticAlgorithm(
    maze=maze,
    start=(0, 0),
    end=(9, 9),
    population_size=10,
    max_generations=15,
    crossover_rate=0.7,
    mutation_rate=0.1
)

In [ ]:
best_path = solver.evolve()